# Two-Patch Tri-Trophic Food Web Simulation

### Importing Packages

In [5]:
import Pkg

Pkg.add("Revise")
Pkg.add("LinearAlgebra")
Pkg.add("Parameters")
Pkg.add("Plots")
Pkg.add("Setfield")
Pkg.add("BifurcationKit")
Pkg.add("OrdinaryDiffEq")

#Supremum Norm
norminf = x -> norm(x, Inf)

   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\T_kolla\.julia\environments\v1.8\Project.

#3 (generic function with 1 method)

### Model Specifications

In [13]:
#Two-patch tri-trophic food web modeled as a vector field
function VF!(dz, z, p, t)
    p₁, p₂, p₃, p₄ = p         #Parameters
    u₁, u₂, u₃, u₄, u₅, u₆ = z #State Variables
    
    #Predator Equations
    dz[1] = u₅/tmat - m*u₁ - p₁*u₁ + s + σ₁*γ*p₂*u₂ - σ₂*p₂*u₁*p₄        #Predator; Littoral
    dz[2] = u₆/tmat - m*u₂ - p₁*u₂ + s - σ₁*p₂*u₂   + σ₂*(1/γ)*p₂*u₁*p₄  #Predator; Palagic

    #Foragers Equations
    dz[3] = r*u₃ - b*u₃*u₃ - h/p₃*u₁*u₃ + i + ϵ*k/p₃*u₃*u₅ + σ₃*γ*p₂*u₄ - σ₄*p₂*u₃       #Forager; Littoral    
    dz[4] = r*u₄ - b*u₄*u₄ - h*u₂*u₄    + i + ϵ*k*u₄*u₆    - σ₃*p₂*u₄   + σ₄*(1/γ)*p₂*u₃ #Forager; Palagic 
    
    #Juveniles Equations
    dz[5] = f*(u₁+γ*u₂) + ϵ*(h/p₃*u₁*u₃ + h*u₂*u₄)  - u₅/tmat - m*u₅ - c*u₁*u₅ - k/p₃*u₃*u₅ + σ₅*γ*p₂*u₆*p₄ - σ₆*p₂*u₅    #Juveniles; Littoral
    dz[6] = -u₆/tmat    - m*u₆ - c*u₂*u₆ - k*u₄*u₆  - σ₅*p₂*u₆*p₄ + σ₆*(1/γ)*p₂*u₅                                        #Juveniles; Palagic
    return dz
end

#Constants 
const m    = 0.250 #Mortality P and J
const s    = 0.050 #Stocking P
const f    = 1.000 #Fecundity
const c    = 0.100 #cannibalism
const k    = 1.000 #Attack rate F on J
const r    = 0.500 #Forage fish growth rate
const b    = 0.050 #Density dependent constant forage fish
const h    = 1.500 #Attack rate P on F
const i    = 0.050 #Stocking F
const ϵ    = 0.250 #Trophic Efficiency
const γ    = 1.000 #Area Ratio
const σ₁   = 1.000 #Dispersal - predator pelagic to littoral
const σ₂   = 1.000 #Dispersal - predator littoral to pelagic
const σ₃   = 1.000 #Dispersal - forage pelagic to littoral
const σ₄   = 1.000 #Dispersal - forage littoral to pelagic
const σ₅   = 1.000 #Dispersal - juvenile pelagic to littoral
const σ₆   = 1.000 #Dispersal - juvenile littoral to pelagic
const tmat = 5.000 #Maturation time for juveniles to adult predators

#State Variables
u₀ = zeros(Float64,6)
u₀[1] = 1.000 #Predator; Littoral
u₀[2] = 1.000 #Predator; Palagic 
u₀[3] = 1.000 #Forager; Littoral
u₀[4] = 1.000 #Forager; Palagic 
u₀[5] = 1.000 #Juveniles; Littoral
u₀[6] = 1.000 #Juveniles; Palagic 

#Parameters
p    = zeros(Float64,4)
p[1] = 0.000 #Fishing effort
p[2] = 1.000 #Dispersal Rate
p[3] = 1.000 #Strength of refuge effect. Larger values = lower predation in littoral
p[4] = 1.000 #

#Timespan
tᵢ  = 0.000 #Initial Time
t_f = 1000.000 #Final Time
tspan = (tᵢ,t_f)

(0.0, 1000.0)

### Simulation

In [19]:
#Defining ODE problem
prob = ODEProblem(
    VF!,   #Vector Field
    u₀,    #Initial Conditions State Variables
    tspan, #Time Interval
    p)     #Initial Conditions Parameters
solution = solve(prob,Tsit5()) #Tsit5 --> Sitff ODE Solver

# Plotting Solution
# Plotting Solution
predator_combined = solution[1,:] .+ solution[2,:]
forager_combined = solution[3,:] .+ solution[4,:]
juvenile_combined = solution[5,:] .+ solution[6,:]

p1 = plot(solution.t, predator_combined,label="Abundance", ylabel="   P", ylims=(0, 11), yguidefontrotation=-90, legend = true)
vline!(p1, [500], linestyle=:dash, linecolor=:red, label="Regime Shift")

p2 = plot(solution.t, forager_combined, ylabel="   F", ylims=(0, 11), yguidefontrotation=-90, legend = false)
vline!(p2, [500], linestyle=:dash, linecolor=:red)

p3 = plot(solution.t, juvenile_combined, ylabel="   J", ylims=(0, 11), yguidefontrotation=-90, legend = false)
vline!(p3, [500], linestyle=:dash, linecolor=:red)

p = plot(p1, p2, p3, layout=(3, 1), xlabel="Time", title = ["Combined Adult Predator Abundance" "Combined Juvenile Predator Abundance" "Combined Forager Abundance"])


LoadError: MethodError: no method matching iterate(::Plots.Plot{Plots.GRBackend}, ::Int64)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, ::Integer) at range.jl:872
[0m  iterate([91m::T[39m, ::Int64) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at dict.jl:712
[0m  iterate([91m::Union{Base.KeySet{<:Any, <:DataStructures.SwissDict}, Base.ValueIterator{<:DataStructures.SwissDict}}[39m, ::Any) at C:\Users\T_kolla\.julia\packages\DataStructures\59MD0\src\swiss_dict.jl:646
[0m  ...